In [3]:
print("Introspection is examining one's own conscious thoughts and feelings. In psychology this is done through observation"
      " of ones mental state. This isn't mediated by external sources of knowledge. This can be applied to AI through "
      "it being able to learn and judge how well its doing based on previous information it has. For example, if it already"
      " has a set of actions that allows it to do a task, if it is asked to learn a similar task it could do it. An AI "
      "should be able to learn more based off what it currently 'knows' without the programmer having to explicitly tell it "
      "how to do everything."
      )

Introspection is examining one's own conscious thoughts and feelings. In psychology this is done through observation of ones mental state. This isn't mediated by external sources of knowledge. This can be applied to AI through it being able to learn and judge how well its doing based on previous information it has. For example, if it already has a set of actions that allows it to do a task, if it is asked to learn a similar task it could do it. An AI should be able to learn more based off what it currently 'knows' without the programmer having to explicitly tell it how to do everything.


In [62]:
"""
This module implements a local search formulation of TSP problem
This solution uses AIMA hill-climbing and simulated annealing implementations to solve it

@author: shendriksen
@version 21feb2019
"""
from search import Problem, hill_climbing, simulated_annealing, \
    exp_schedule, genetic_search
from random import randrange
import time
import math

class TSP(Problem):
    """
    State: a complete list of all the cities in the problem that represents the path the salesman would take
    Move: repositioning any city to a different location in the list
    """
    def __init__(self, initial, distances):
        self.initial = initial
        self.distances = distances
    
    def actions(self, state):
        """Returns the list of all valid city swaps"""
        actions = []
        for city_a in range(1, len(self.initial)):
            for city_b in [x for x in range(1, len(self.initial)) if x != state[city_a]]:
                actions.append([city_a, city_b])
        return actions

    def value(self, current_path):
        """Returns the value of a given state - in this case, the length of the path (negative)"""
        total_distance = 0
    
        for i in range(len(initial) - 1):
            city_1 = current_path[i]
            city_2 = current_path[int(i + 1)]
            
            # because the dictionary is ordered with x < y, this checks to see which order it should put a and b 
            #   into the tuple that is searched in the dictionary
            if city_1 < city_2:
                total_distance = total_distance + distances[(city_1, city_2)]
            else:
                total_distance = total_distance + distances[(city_2, city_1)]
        
        total_distance = total_distance + distances[(0, current_path[len(self.initial) - 1])]
        
        distance_val = total_distance * -1
        return distance_val

    def result(self, state, move):
        """Makes the given move on a copy of the given state."""
        new_state = state[:]
        
        city_a_idx = state.index(move[0])
        city_b_idx = state.index(move[1])
        
        new_state[city_a_idx] = move[1]
        new_state[city_b_idx] = move[0]
        
        return new_state


if __name__ == '__main__':
    num_cities = 50
    max_dist = 10
    distances = {}
    initial = []
    
    # creates the initial list of all the cities and a dictionary of distances between cites {(tuple: int)}
    for a in range(num_cities):
        initial.append(a)
        
        for b in range(a + 1, num_cities):
            distances.update({(a, b): randrange(1, max_dist)})
    
    p = TSP(initial, distances)
    
    hill_solution = hill_climbing(p)
    print('Hill-climbing solution       \n\tpath: ' + str(hill_solution)
        + '\n\tdistance: ' + str(p.value(hill_solution) * (-1)))
    
    annealing_solution = simulated_annealing(p)
    print('Simulated Annealing solution       \n\tpath: ' + str(annealing_solution)
        + '\n\tdistance: ' + str(p.value(annealing_solution) * (-1)))
    
    print("\nThe local search algorithm that worked best for me was hill-climbing. This is "
          "because when you are doing random swaps of cities you wouldn't want to ever increase "
          "your value which hill-climbing does. This means that every step it takes it is getting "
          "the result closer to the best solution. Simulated annealing however can potentially select "
          "a worse step which means it won't always get closer to the shortest path.")

Hill-climbing solution       
	path: [0, 24, 25, 31, 4, 20, 13, 34, 26, 22, 47, 11, 44, 6, 14, 15, 16, 17, 49, 19, 5, 43, 27, 23, 37, 2, 8, 9, 28, 21, 32, 3, 30, 33, 7, 35, 36, 1, 38, 12, 40, 29, 42, 41, 39, 45, 46, 10, 48, 18]
	distance: 84


Simulated Annealing solution       
	path: [0, 10, 39, 41, 32, 5, 8, 48, 26, 35, 18, 30, 13, 46, 19, 45, 24, 16, 34, 17, 29, 20, 40, 12, 7, 31, 21, 44, 11, 9, 28, 6, 47, 1, 3, 22, 42, 37, 38, 43, 36, 15, 25, 49, 33, 14, 23, 27, 2, 4]
	distance: 228
The local search algorithm that worked best for me was hill-climbing. This is because when you are doing random swaps of cities you wouldn't want to ever increase your value which hill-climbing does. This means that every step it takes it is getting the result closer to the best solution. Simulated annealing however can potentially select a worse step which means it won't always get closer to the shortest path.


In [63]:
"""
This module implements a local search formulation of TSP problem
This solution uses AIMA hill-climbing and simulated annealing implementations to solve it

@author: shendriksen
@version 21feb2019
"""
from csp import parse_neighbors, backtracking_search, CSP


def print_result(result):
    """Prints the results in a formatted way"""
    for key in result:
        print("Class: " + key)
        details = result[key].split()
        print("\tProfessor: " + details[0])
        print("\tTime: " + details[1])
        print("\tRoom: " + details[2])


def set_valid_domains(variable, neighbors, times, classrooms):
    """Creates a list of valid time/classroom combinations for each class (accounts for valid faculty)"""
    domain_list = []
    
    for f in neighbors[variable]:
        for t in times:
            for c in classrooms:
                domain_list.append(f + ' ' + t + ' ' + c)
                
    return domain_list


def cs_classes_scheduler():
    """Return an instance of a CS Class Problem"""
    variables = 'cs108 cs112 cs212 cs214 cs344 cs232 cs384'.split()
    faculty = 'schuurman adams vanderlinden plantinga norman'.split()
    times = '800 900 1030 1130 1230'.split()
    classrooms = 'nh253 sb382'.split()
    domains = {}

    neighbors = parse_neighbors("""norman: cs108; adams: cs112; plantinga: cs212; adams: cs214; 
        vanderlinden: cs344; adams: cs232; schuurman: cs384""", variables)
    
    for var in variables:
        domains[var] = set_valid_domains(var, neighbors, times, classrooms)
    
    for A in variables:
        for B in variables:
            if A != B:
                if B not in neighbors[A]:
                    neighbors[A].append(B)
                if A not in neighbors[B]:
                    neighbors[B].append(A)
                        
    def classes_constraint(A, a, B, b, recurse=0):
        """Checks that all constraints are met in the proposed schedule"""
        if A == 'norman' and B == 'cs108':
            return same
        if A == 'adams' and B == 'cs112':
            return same
        if A == 'plantinga' and B == 'cs212':
            return same
        if A == 'adams' and B == 'cs214':
            return same
        if A == 'vanderlinden' and B == 'cs344':
            return same
        if A == 'adams' and B == 'cs232':
            return same
        if A == 'schuurman' and B == 'cs384':
            return same
        
        if a.split()[0] == b.split()[0] and a.split()[1] == b. split()[1]:
            return False
        if a.split()[1] == b.split()[1] and a.split()[2] == b. split()[2]:
            return False
        
        if recurse == 0:
            return classes_constraint(B, b, A, a, 1)
        
        return True
    
    return CSP(variables, domains, neighbors, classes_constraint)
        

if __name__ == '__main__':
    puzzle = cs_classes_scheduler()
    backtracking_result = backtracking_search(puzzle)
    
    if puzzle.goal_test(puzzle.infer_assignment()):
        print("Solution (using back tracking):\n")
        print_result(backtracking_result)
    else:
        print("failed...")
        print(puzzle.curr_domains)
        puzzle.display(puzzle.infer_assignment())
        
    print("\nTo solve this problem I chose back-tracking. This is because backtracking keeps pursuing "
          "solutions but as soon as it swaps something (in this case a class) that doesn't fulfill the "
          "constraints given, it will abandon that as a potential solution.")


Solution (using back tracking):

Class: cs344
	Professor: vanderlinden
	Time: 1030
	Room: nh253
Class: cs232
	Professor: adams
	Time: 1030
	Room: sb382
Class: cs112
	Professor: adams
	Time: 800
	Room: sb382
Class: cs384
	Professor: schuurman
	Time: 1130
	Room: nh253
Class: cs212
	Professor: plantinga
	Time: 900
	Room: nh253
Class: cs214
	Professor: adams
	Time: 900
	Room: sb382
Class: cs108
	Professor: norman
	Time: 800
	Room: nh253

To solve this problem I chose back-tracking. This is because backtracking keeps pursuing solutions but as soon as it swaps something (in this case a class) that doesn't fulfill the constraints given, it will abandon that as a potential solution.
